# Equity Option Pricing

This notebook replicates the QuantLib `EquityOption` example, demonstrating various pricing engines for European, Bermudan, and American options.

**Engines covered:**
- Analytic: Black-Scholes, Heston, Bates, Black-Vasicek
- Approximation: Barone-Adesi-Whaley, Bjerksund-Stensland
- Numerical: Integral, Finite Differences, Binomial Trees
- Monte Carlo: European MC, American MC (Longstaff-Schwartz)

In [ ]:
import pyquantlib as ql
import matplotlib.pyplot as plt

print(f"PyQuantLib {ql.__version__} (QuantLib {ql.QL_VERSION})")

## Setup

Market data matching the original QuantLib example.

In [2]:
# Dates
calendar = ql.TARGET()
today = ql.Date(15, ql.May, 1998)
settlement = ql.Date(17, ql.May, 1998)
maturity = ql.Date(17, ql.May, 1999)
ql.Settings.instance().evaluationDate = today

# Option parameters
option_type = ql.OptionType.Put
underlying = 36.0
strike = 40.0
dividend_yield = 0.00
risk_free_rate = 0.06
volatility = 0.20

print(f"Option type:    {option_type}")
print(f"Maturity:       {maturity}")
print(f"Underlying:     {underlying}")
print(f"Strike:         {strike}")
print(f"Risk-free rate: {risk_free_rate:.2%}")
print(f"Dividend yield: {dividend_yield:.2%}")
print(f"Volatility:     {volatility:.2%}")

Option type:    OptionType.Put
Maturity:       May 17th, 1999
Underlying:     36.0
Strike:         40.0
Risk-free rate: 6.00%
Dividend yield: 0.00%
Volatility:     20.00%


## Term Structures and Process

In [3]:
dc = ql.Actual365Fixed()

# Market data
spot = ql.SimpleQuote(underlying)
flat_rate = ql.FlatForward(settlement, risk_free_rate, dc)
flat_div = ql.FlatForward(settlement, dividend_yield, dc)
flat_vol = ql.BlackConstantVol(settlement, calendar, volatility, dc)

# Black-Scholes-Merton process
bsm_process = ql.BlackScholesMertonProcess(spot, flat_div, flat_rate, flat_vol)

## Options

Create European, Bermudan, and American options with the same payoff.

In [4]:
payoff = ql.PlainVanillaPayoff(option_type, strike)

# European exercise
european_exercise = ql.EuropeanExercise(maturity)
european_option = ql.VanillaOption(payoff, european_exercise)

# Bermudan exercise (quarterly)
bermudan_dates = [settlement + ql.Period(3 * i, ql.Months) for i in range(1, 5)]
bermudan_exercise = ql.BermudanExercise(bermudan_dates)
bermudan_option = ql.VanillaOption(payoff, bermudan_exercise)

# American exercise
american_exercise = ql.AmericanExercise(settlement, maturity)
american_option = ql.VanillaOption(payoff, american_exercise)

## Results Table

Helper function to collect and display results.

In [5]:
results = []

def add_result(method, european=None, bermudan=None, american=None):
    results.append({
        "Method": method,
        "European": f"{european:.6f}" if european else "N/A",
        "Bermudan": f"{bermudan:.6f}" if bermudan else "N/A",
        "American": f"{american:.6f}" if american else "N/A",
    })

def show_results():
    print(f"{'Method':<35} {'European':<14} {'Bermudan':<14} {'American':<14}")
    print("-" * 77)
    for r in results:
        print(f"{r['Method']:<35} {r['European']:<14} {r['Bermudan']:<14} {r['American']:<14}")

## Analytic Engines

In [6]:
# Black-Scholes
european_option.setPricingEngine(ql.AnalyticEuropeanEngine(bsm_process))
add_result("Black-Scholes", european=european_option.NPV())

In [7]:
# Black-Vasicek (stochastic interest rates)
vasicek = ql.Vasicek(r0=risk_free_rate, a=0.3, b=0.3, sigma=0.15)
european_option.setPricingEngine(
    ql.AnalyticBlackVasicekEngine(bsm_process, vasicek, correlation=0.5)
)
add_result("Black-Vasicek", european=european_option.NPV())

In [8]:
# Heston (stochastic volatility)
heston_process = ql.HestonProcess(
    flat_rate, flat_div, spot,
    v0=volatility**2,
    kappa=1.0,
    theta=volatility**2,
    sigma=0.001,
    rho=0.0,
)
heston_model = ql.HestonModel(heston_process)
european_option.setPricingEngine(ql.AnalyticHestonEngine(heston_model))
add_result("Heston semi-analytic", european=european_option.NPV())

In [9]:
# Bates (stochastic volatility + jumps)
bates_process = ql.BatesProcess(
    ql.YieldTermStructureHandle(flat_rate),
    ql.YieldTermStructureHandle(flat_div),
    ql.QuoteHandle(spot),
    volatility**2,  # v0
    1.0,            # kappa
    volatility**2,  # theta
    0.001,          # sigma
    0.0,            # rho
    1e-14,          # lambda (jump intensity)
    1e-14,          # nu (mean jump size)
    1e-14,          # delta (jump size vol)
)
bates_model = ql.BatesModel(bates_process)
european_option.setPricingEngine(ql.BatesEngine(bates_model))
add_result("Bates semi-analytic", european=european_option.NPV())

## American Approximations

In [10]:
# Barone-Adesi-Whaley
american_option.setPricingEngine(
    ql.BaroneAdesiWhaleyApproximationEngine(bsm_process)
)
add_result("Barone-Adesi/Whaley", american=american_option.NPV())

In [11]:
# Bjerksund-Stensland
american_option.setPricingEngine(
    ql.BjerksundStenslandApproximationEngine(bsm_process)
)
add_result("Bjerksund/Stensland", american=american_option.NPV())

## Integral Engine

In [12]:
european_option.setPricingEngine(ql.IntegralEngine(bsm_process))
add_result("Integral", european=european_option.NPV())

## Finite Differences

In [13]:
time_steps = 801
fd_engine = ql.FdBlackScholesVanillaEngine(bsm_process, time_steps, time_steps - 1)

european_option.setPricingEngine(fd_engine)
bermudan_option.setPricingEngine(fd_engine)
american_option.setPricingEngine(fd_engine)

add_result(
    "Finite differences",
    european=european_option.NPV(),
    bermudan=bermudan_option.NPV(),
    american=american_option.NPV(),
)

## Binomial Trees

In [14]:
tree_types = [
    ("jr", "Binomial Jarrow-Rudd"),
    ("crr", "Binomial Cox-Ross-Rubinstein"),
    ("eqp", "Additive equiprobabilities"),
    ("trigeorgis", "Binomial Trigeorgis"),
    ("tian", "Binomial Tian"),
    ("lr", "Binomial Leisen-Reimer"),
    ("joshi", "Binomial Joshi"),
]

for tree_type, name in tree_types:
    engine = ql.BinomialVanillaEngine(bsm_process, tree_type, time_steps)
    european_option.setPricingEngine(engine)
    bermudan_option.setPricingEngine(engine)
    american_option.setPricingEngine(engine)
    add_result(
        name,
        european=european_option.NPV(),
        bermudan=bermudan_option.NPV(),
        american=american_option.NPV(),
    )

## Monte Carlo

In [15]:
mc_seed = 42

# MC (crude) - pseudorandom
european_option.setPricingEngine(
    ql.MCEuropeanEngine(
        bsm_process,
        rngType="pseudorandom",
        timeSteps=1,
        requiredTolerance=0.02,
        seed=mc_seed,
    )
)
add_result("MC (crude)", european=european_option.NPV())

In [16]:
# QMC (Sobol) - low discrepancy
european_option.setPricingEngine(
    ql.MCEuropeanEngine(
        bsm_process,
        rngType="lowdiscrepancy",
        timeSteps=1,
        requiredSamples=32768,
    )
)
add_result("QMC (Sobol)", european=european_option.NPV())

In [17]:
# MC American (Longstaff-Schwartz)
american_option.setPricingEngine(
    ql.MCAmericanEngine(
        bsm_process,
        rngType="pseudorandom",
        timeSteps=100,
        antitheticVariate=True,
        calibrationSamples=4096,
        requiredTolerance=0.02,
        seed=mc_seed,
    )
)
add_result("MC (Longstaff-Schwartz)", american=american_option.NPV())

## Results Summary

In [18]:
show_results()

Method                              European       Bermudan       American      
-----------------------------------------------------------------------------
Black-Scholes                       3.844308       N/A            N/A           
Black-Vasicek                       3.818577       N/A            N/A           
Heston semi-analytic                3.844306       N/A            N/A           
Bates semi-analytic                 3.844306       N/A            N/A           
Barone-Adesi/Whaley                 N/A            N/A            4.459628      
Bjerksund/Stensland                 N/A            N/A            4.453064      
Integral                            3.844309       N/A            N/A           
Finite differences                  3.844330       4.360765       4.486113      
Binomial Jarrow-Rudd                3.844132       4.361174       4.486552      
Binomial Cox-Ross-Rubinstein        3.843504       4.360861       4.486415      
Additive equiprobabilities     

## Notes

- **European options**: All analytic engines give the same Black-Scholes price (~3.844) when using equivalent parameters
- **American options**: Early exercise premium makes American put worth more (~4.48) than European
- **Bermudan options**: Value between European and American
- **QdFpAmericanEngine**: Not included due to Windows stability issues